In [2]:
import gurobipy as gb
from gurobipy import GRB

import pandas as pd

df_demand = pd.read_csv('data/df_demand.csv', index_col=0)
df_demand.set_index('STORE', inplace=True)
df_demand.head()

,DEMAND
STORE,
D1,244
D2,172
D3,124
D4,90
D5,158


In [3]:
df_inbound = pd.read_csv('data/df_inbound_price.csv', index_col=0)
df_outbound = pd.read_csv('data/df_outbound_price.csv', index_col=0)
df_outbound = df_outbound.set_index('from')
df_outbound.head()

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,...,S191,S192,S193,S194,S195,S196,S197,S198,S199,S200
from,,,,,,,,,,,,,,,,,,,,,
D1,2.30,4.23,2.26,3.38,1.59,2.01,5.32,6.63,2.38,6.62,...,5.86,8.30,3.02,1.01,2.77,2.96,3.53,8.60,2.77,7.06
D2,5.31,2.18,8.52,8.34,4.59,1.04,1.89,6.45,8.35,3.32,...,7.54,2.11,4.33,1.54,4.75,7.84,8.21,4.51,3.27,3.13


In [4]:
df_inbound.head()

,D1,D2
P1,3.0,5.0
P2,2.3,6.6


In [5]:
n_p, n_dc = df_inbound.shape
n_s = df_demand.shape[0]
n_p, n_dc, n_s

(2, 2, 200)

In [6]:
m = gb.Model('supply_chain.lp')

I = m.addVars(n_p, n_dc, vtype=GRB.INTEGER, name='Inbound')
O = m.addVars(n_dc, n_s, vtype=GRB.INTEGER, name='Outbound')
ic = df_inbound.to_numpy()
oc = df_outbound.to_numpy()
demand = df_demand.to_numpy().flatten()

m.update()

Restricted license - for non-production use only - expires 2025-11-24


In [7]:
objective = gb.quicksum(ic[i, j] * I[i, j] for i in range(n_p) for j in range(n_dc))
objective += gb.quicksum(oc[j, k] * O[j, k] for j in range(n_dc) for k in range(n_s))

m.setObjective(objective, GRB.MINIMIZE)
m.update()

In [8]:
m.addConstrs((gb.quicksum(I[i, j] for i in range(n_p)) == gb.quicksum(O[j, k] for k in range(n_s))) for j in range(n_dc))

m.addConstrs((gb.quicksum(O[j, k] for j in range(n_dc)) >= demand[k]) for k in range(n_s))

m.update()

In [9]:
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: AMD EPYC 7763 64-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 202 rows, 404 columns and 804 nonzeros
Model fingerprint: 0xe2353170
Variable types: 0 continuous, 404 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 3e+02]
Found heuristic solution: objective 290103.22000
Presolve removed 202 rows and 404 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 2: 217189 290103 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.171893200000e+05, best bound 2.171893200000e+05, gap 0.0000%


In [20]:
import numpy as np
np.array([x.X for x in I.values()]).reshape(-1, n_dc)

array([[   -0.,  6232.],
       [25574.,    -0.]])